<a href="https://www.kaggle.com/code/dimitar0dimov/knn-classifier-for-london-housing-average-prices?scriptVersionId=102580124" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [64]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/housing-in-london/housing_in_london_monthly_variables.csv
/kaggle/input/housing-in-london/housing_in_london_yearly_variables.csv


In [65]:
data_monthly = pd.read_csv('/kaggle/input/housing-in-london/housing_in_london_monthly_variables.csv')
data_yearly = pd.read_csv('/kaggle/input/housing-in-london/housing_in_london_yearly_variables.csv')

data_monthly["date"] = pd.to_datetime(data_monthly["date"])
data_yearly["date"] = pd.to_datetime(data_yearly["date"])

In [66]:
data_yearly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   code               1071 non-null   object        
 1   area               1071 non-null   object        
 2   date               1071 non-null   datetime64[ns]
 3   median_salary      1049 non-null   float64       
 4   life_satisfaction  352 non-null    float64       
 5   mean_salary        1071 non-null   object        
 6   recycling_pct      860 non-null    object        
 7   population_size    1018 non-null   float64       
 8   number_of_jobs     931 non-null    float64       
 9   area_size          666 non-null    float64       
 10  no_of_houses       666 non-null    float64       
 11  borough_flag       1071 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 100.5+ KB


Drop the emptiest column life_satisfaction as well as borough_flag because the latter is just ones and zeroes.

In [67]:
data_yearly = data_yearly.drop(["life_satisfaction", "borough_flag"], axis=1)
data_yearly.isnull().sum()

code                 0
area                 0
date                 0
median_salary       22
mean_salary          0
recycling_pct      211
population_size     53
number_of_jobs     140
area_size          405
no_of_houses       405
dtype: int64

Let's drop the rows with zeroes for specific columns first and then the remaining.

In [68]:
data_yearly = data_yearly.dropna(subset=["median_salary","population_size","number_of_jobs"],how="any")
data_yearly.isnull().sum()

code                 0
area                 0
date                 0
median_salary        0
mean_salary          0
recycling_pct      114
population_size      0
number_of_jobs       0
area_size          299
no_of_houses       299
dtype: int64

In [69]:
data_yearly = data_yearly.dropna()
data_yearly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 102 to 1013
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   code             621 non-null    object        
 1   area             621 non-null    object        
 2   date             621 non-null    datetime64[ns]
 3   median_salary    621 non-null    float64       
 4   mean_salary      621 non-null    object        
 5   recycling_pct    621 non-null    object        
 6   population_size  621 non-null    float64       
 7   number_of_jobs   621 non-null    float64       
 8   area_size        621 non-null    float64       
 9   no_of_houses     621 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 53.4+ KB


In [70]:
data_yearly.head(10)

,code,area,date,median_salary,mean_salary,recycling_pct,population_size,number_of_jobs,area_size,no_of_houses
102,E09000001,city of london,2001-12-01,39104.0,62819,0,7359.0,339000.0,315.0,5009.0
103,E09000002,barking and dagenham,2001-12-01,22323.0,26050,3,165654.0,54000.0,3780.0,68298.0
104,E09000003,barnet,2001-12-01,20916.0,26068,8,319481.0,138000.0,8675.0,130515.0
105,E09000004,bexley,2001-12-01,20217.0,23559,20,218757.0,75000.0,6429.0,91606.0
106,E09000005,brent,2001-12-01,21878.0,24164,7,269620.0,116000.0,4323.0,101427.0
107,E09000006,bromley,2001-12-01,15684.0,19749,14,296218.0,114000.0,15013.0,128717.0
108,E09000007,camden,2001-12-01,27386.0,40033,12,202567.0,294000.0,2179.0,93112.0
109,E09000008,croydon,2001-12-01,20889.0,23994,12,335112.0,156000.0,8650.0,141300.0
110,E09000009,ealing,2001-12-01,23862.0,29516,13,307276.0,134000.0,5554.0,120331.0
111,E09000010,enfield,2001-12-01,24136.0,27605,13,277266.0,111000.0,8220.0,112948.0


Let's pre-process the monthly dataset and combine it with yearly so we can use all of the features to predict average prices.

In [71]:
data_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           13549 non-null  datetime64[ns]
 1   area           13549 non-null  object        
 2   average_price  13549 non-null  int64         
 3   code           13549 non-null  object        
 4   houses_sold    13455 non-null  float64       
 5   no_of_crimes   7439 non-null   float64       
 6   borough_flag   13549 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 741.1+ KB


In [72]:
data_monthly.isnull().sum()

date                0
area                0
average_price       0
code                0
houses_sold        94
no_of_crimes     6110
borough_flag        0
dtype: int64

In [73]:
data_monthly = data_monthly.dropna(subset=["houses_sold"],how="any")
data_monthly = data_monthly.drop(["no_of_crimes","borough_flag"],axis=1)
data_monthly.isnull().sum()

date             0
area             0
average_price    0
code             0
houses_sold      0
dtype: int64

In [74]:
data_monthly["year"] = pd.DatetimeIndex(data_monthly["date"]).year
data_monthly["month"] = pd.DatetimeIndex(data_monthly["date"]).month
data_monthly["day"] = pd.DatetimeIndex(data_monthly["date"]).day
data_monthly = data_monthly.drop(["date"],axis=1)

As we can see, the monthly dataset is much more granular than the yearly. We need to convert the monthly dataset into a yearly so we can join the two "yearly" datasets afterwords. 

We will take the average values for both average_price and houses_sold. 

In [75]:
df3 = data_monthly.groupby(['year','code'])["average_price"].mean().reset_index()
df4 = data_monthly.groupby(['year','code'])["houses_sold"].mean().reset_index()
data_monthly_s = pd.merge(df3,df4,on=["year","code"])

In [76]:
data_yearly["year"] = pd.DatetimeIndex(data_yearly["date"]).year
data_yearly["month"] = pd.DatetimeIndex(data_yearly["date"]).month
data_yearly["day"] = pd.DatetimeIndex(data_yearly["date"]).day
data_yearly = data_yearly.drop(["date"], axis=1)

Ready to merge the two datasets and apply the ML models.

In [77]:
combined = pd.merge(data_yearly,data_monthly_s,on=["year","code"])

In [78]:
combined.head(10)

,code,area,median_salary,mean_salary,recycling_pct,population_size,number_of_jobs,area_size,no_of_houses,year,month,day,average_price,houses_sold
0,E09000001,city of london,39104.0,62819,0,7359.0,339000.0,315.0,5009.0,2001,12,1,250625.583333,30.583333
1,E09000002,barking and dagenham,22323.0,26050,3,165654.0,54000.0,3780.0,68298.0,2001,12,1,88664.000000,266.916667
2,E09000003,barnet,20916.0,26068,8,319481.0,138000.0,8675.0,130515.0,2001,12,1,185563.333333,566.916667
3,E09000004,bexley,20217.0,23559,20,218757.0,75000.0,6429.0,91606.0,2001,12,1,116527.083333,439.166667
4,E09000005,brent,21878.0,24164,7,269620.0,116000.0,4323.0,101427.0,2001,12,1,157287.333333,393.750000
5,E09000006,bromley,15684.0,19749,14,296218.0,114000.0,15013.0,128717.0,2001,12,1,162131.833333,626.833333
6,E09000007,camden,27386.0,40033,12,202567.0,294000.0,2179.0,93112.0,2001,12,1,278593.916667,299.166667
7,E09000008,croydon,20889.0,23994,12,335112.0,156000.0,8650.0,141300.0,2001,12,1,132509.333333,576.000000
8,E09000009,ealing,23862.0,29516,13,307276.0,134000.0,5554.0,120331.0,2001,12,1,170190.500000,503.083333
9,E09000010,enfield,24136.0,27605,13,277266.0,111000.0,8220.0,112948.0,2001,12,1,135713.166667,589.416667


In [79]:
combined = combined.drop(["area","month","day"],axis=1)

Convert the string data entries from the code column into integers.

In [80]:
from sklearn.preprocessing import LabelEncoder
LB = LabelEncoder()
combined["codeEnc"] = LB.fit_transform(combined["code"])
combined = combined.drop(["code"],axis=1)

I did this analysis on my on machine first and found out that there are some "#" entries in the mean_salary column. How? After I ran the first regression model it came back with an error "could not convert string to float". 

So I searched for an elegant solution to iterate through the dataframe and find where the "#" is. The following code was published on Stack Overflow. 


In [81]:
mask = np.column_stack([combined[col].astype(str).str.contains(r"#", na=False) for col in combined])
combined.loc[mask.any(axis=1)]

,median_salary,mean_salary,recycling_pct,population_size,number_of_jobs,area_size,no_of_houses,year,average_price,houses_sold,codeEnc
11,24095.0,#,1,207246.0,108000.0,1905.0,87208.0,2001,153406.416667,274.083333,11
177,25715.0,#,32,300963.0,126000.0,15013.0,131387.0,2006,249691.750000,697.250000,5
526,31479.0,#,39,382304.0,149000.0,8650.0,154559.0,2016,355846.666667,442.666667,7
610,31182.0,#,39,206186.0,108000.0,3762.0,84297.0,2018,512293.916667,190.250000,23


In [82]:
combined = combined[combined["mean_salary"] != '#']

I will try linear regression first and then knn classifier. 

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics  
# Split df into X and y
y = combined['average_price']
X = combined.drop(['average_price'], axis=1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65)

regressor = LinearRegression()  
regressor.fit(X_train, y_train) 

y_pred = regressor.predict(X_train) 
print('Mean Absolute Error (Train set):',metrics.mean_absolute_error(y_train, y_pred)) 
rmse = np.sqrt(np.mean((y_train - y_pred)**2))
print('Root Mean Absolute Error (Train set):',rmse)

Mean Absolute Error (Train set): 80916.14761961071
Root Mean Absolute Error (Train set): 127333.86662037234


In [84]:
regressor.score(X_train, y_train)

0.5619808509173576

In [85]:
y_pred = regressor.predict(X_test) 
print('Mean Absolute Error (Test set):',metrics.mean_absolute_error(y_test, y_pred)) 
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print('Root Mean Sqaured Error (Test set):',rmse)

Mean Absolute Error (Test set): 80777.79672979256
Root Mean Sqaured Error (Test set): 120147.98700155228


In [86]:
regressor.score(X_test, y_test)

0.5160938894976626

In [87]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X_train, y_train)
train_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
print('Root Mean Squared Error (Train set):',rmse)

Root Mean Squared Error (Train set): 51742.23446521706


In [88]:
knn_model.score(X_train,y_train)

0.9276738076174711

In [89]:
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
print('Root Mean Squared Error (Test set):',rmse)

Root Mean Squared Error (Test set): 60172.91515221299


In [90]:
knn_model.score(X_test,y_test)

0.8786247290855562

Linear Regression score: **54.4%** \
KNeighbours Classifier score: **87.8%**

Let's see if scaling the values will improve the knn classifier at all.

In [91]:
scaler = StandardScaler()
combined_sc = scaler.fit_transform(combined)
scaled_features_df = pd.DataFrame(combined_sc, index=combined.index, columns=combined.columns)
scaled_features_df

,median_salary,mean_salary,recycling_pct,population_size,number_of_jobs,area_size,no_of_houses,year,average_price,houses_sold,codeEnc
0,1.472940,2.193591,-2.397051,-0.224378,-0.162651,-0.177424,-0.221648,-1.642570,-0.477231,-0.209388,-1.688072
1,-1.275889,-0.977441,-2.133026,-0.206518,-0.224214,-0.175876,-0.205099,-1.642570,-1.348230,-0.191615,-1.589066
2,-1.506364,-0.975889,-1.692984,-0.189162,-0.206069,-0.173688,-0.188830,-1.642570,-0.827124,-0.169055,-1.490061
3,-1.620865,-1.192270,-0.636884,-0.200527,-0.219678,-0.174692,-0.199004,-1.642570,-1.198388,-0.178662,-1.391055
4,-1.348783,-1.140093,-1.780993,-0.194788,-0.210821,-0.175633,-0.196436,-1.642570,-0.979187,-0.182077,-1.292049
...,...,...,...,...,...,...,...,...,...,...,...
616,0.030464,-0.388839,0.419216,-0.193989,-0.216869,-0.175830,-0.196017,1.655461,0.545810,-0.194053,1.282100
617,0.718941,0.684183,-0.372859,-0.188373,-0.204125,-0.175991,-0.184738,1.655461,1.383618,-0.182040,1.381106
618,2.113585,2.277504,-0.460867,-0.196401,-0.068471,-0.176580,-0.190400,1.655461,3.660455,-0.198603,1.480112
619,1.316014,1.314785,0.507225,0.779844,1.092149,-0.106292,0.706938,1.655461,0.744764,0.351047,1.579118


In [92]:
y = scaled_features_df['average_price']
X = scaled_features_df.drop(['average_price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X_train, y_train)

test_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, test_preds)
rmse = sqrt(mse)
rmse

0.31432722510127864

In [93]:
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
rmse

0.4301642682871334

In [94]:
knn_model.score(X_test,y_test)

0.8504521763067618

Nope, not really. 